In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.layers import *
from tensorflow.keras.models import *

2023-04-03 11:02:41.894210: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 11:02:42.020505: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-03 11:02:42.047509: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a suppor

# DeepLab V3+

## Encoder 생성
- ResNet50을 기반모델로 생성

In [21]:
from tensorflow.keras.applications.resnet50 import ResNet50

inputs = keras.layers.Input(shape=input_shape)

def encoder(inputs):
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)

    # ResNet50에서의 layer 이름을 기반으로, 해당 layer까지의 output tensor를 저장
    x1 = base_model.get_layer('conv4_block6_out').output
    x2 = base_model.get_layer('conv3_block4_out').output
    x3 = base_model.get_layer('conv2_block3_out').output
    x4 = base_model.get_layer('conv1_relu').output

    return x1, x2, x3, x4

x1, x2, x3, x4 = encoder(inputs)

### ASPP(Atrous Spatial Pyramid Pooling)
##### Atrous convolution는 dilated convolution과 거의 비슷하다.
- Dilated Convolution은 필터에 구멍을 뚫어서 연산량을 줄이는 방식으로 연산을 수행(https://github.com/vdumoulin/conv_arithmetic/blob/master/gif/dilation.gif)
- Atrous Convolution은 필터의 크기를 확장시키고 zero padding을 추가하여 연산량을 줄이는 방식으로 연산을 수행한다.

In [22]:
def aspp(x, output_stride):
    # global average pooling
    pool_size = (tf.shape(x)[1], tf.shape(x)[2])
    x = layers.AveragePooling2D(pool_size=pool_size)(x)
    x = layers.Conv2D(256, 1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # 1x1 convolution
    b = layers.Conv2D(256, 1, dilation_rate=1, padding='same', use_bias=False)(x)
    b = layers.BatchNormalization()(b)
    b = layers.ReLU()(b)

    # 3x3 convolutions with different dilation rates
    r1 = layers.Conv2D(256, 3, dilation_rate=6, padding='same', use_bias=False)(x)
    r1 = layers.BatchNormalization()(r1)
    r1 = layers.ReLU()(r1)

    r2 = layers.Conv2D(256, 3, dilation_rate=12, padding='same', use_bias=False)(x)
    r2 = layers.BatchNormalization()(r2)
    r2 = layers.ReLU()(r2)

    # Block 3
    r3 = layers.SeparableConv2D(256, (3, 3), padding='same', name='block3_sepconv1')(r2)
    r3 = layers.BatchNormalization(name='block3_sepconv1_bn')(r3)
    r3 = layers.ReLU(name='block3_sepconv2_act')(r3)
    r3 = layers.SeparableConv2D(256, (3, 3), padding='same', name='block3_sepconv2')(r3)
    r3 = layers.BatchNormalization(name='block3_sepconv2_bn')(r3)
    r3 = layers.ReLU(name='block3_sepconv3_act')(r3)
    r3 = layers.SeparableConv2D(256, (3, 3), padding='same', name='block3_sepconv3')(r3)
    r3 = layers.BatchNormalization(name='block3_sepconv3_bn')(r3)
    r3 = layers.ReLU()(r3)

    # Block 4
    x = layers.Conv2D(4 * 256, (1, 1), padding='same', strides=(2, 2),
    name='block4_conv1')(r3)
    x = layers.BatchNormalization(name='block4_conv1_bn')(x)

    prev_res = layers.Conv2D(4 * 256, (1, 1), padding='same', strides=(2, 2),
    name='block2_conv1')(r1)
    prev_res = layers.BatchNormalization(name='block2_conv1_bn')(prev_res)
    x = layers.Add(name='block4_add')([x, prev_res])

    # Block 5
    x = layers.SeparableConv2D(512, (3, 3), padding='same', name='block5_sepconv1')(x)
    x = layers.BatchNormalization(name='block5_sepconv1_bn')(x)
    x = layers.ReLU(name='block5_sepconv2_act')(x)
    x = layers.SeparableConv2D(512, (3, 3), padding='same', name='block5_sepconv2')(x)
    x = layers.BatchNormalization(name='block5_sepconv2_bn')(x)
    x = layers.ReLU(name='block5_sepconv3_act')(x)
    x = layers.SeparableConv2D(512, (3, 3), padding='same', name='block5_sepconv3')(x)
    x = layers.BatchNormalization(name='block5_sepconv3_bn')(x)
    x = layers.ReLU()(x)

    # Block 6
    x = layers.AveragePooling2D((7, 7), name='avg_pool')(x)
    x = layers.Conv2D(classes, (1, 1), padding='same', name='prediction')(x)
    x = layers.Activation('softmax', name='softmax')(x)


    model = Model(img_input, x, name='deeplabv3plus')

    return model

In [23]:
input_shape = (224, 224, 3); num_classes = 10

# 모델 생성
model = DeConvNet(input_shape, num_classes)

model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_572 (Conv2D)         (None, 224, 224, 64)      1792      
                                                                 
 batch_normalization_500 (Ba  (None, 224, 224, 64)     256       
 tchNormalization)                                               
                                                                 
 activation_322 (Activation)  (None, 224, 224, 64)     0         
                                                                 
 conv2d_573 (Conv2D)         (None, 224, 224, 64)      36928     
                                                                 
 batch_normalization_501 (Ba  (None, 224, 224, 64)     256       
 tchNormalization)                                        